In [ ]:
import pandas as pd
import dateutil

In [ ]:
def getUserList():
    userTypes = ['AS_STUDENT', 'AS_TEACHERS', 'AS_NON-EDITING-TEACHERS']
    userDF = pd.DataFrame()
    for userType in userTypes:
        userTypeDF = pd.read_csv(userType + '.csv')
        userTypeDF['User Type'] = userType
        userDF = userDF.append(userTypeDF, ignore_index=True, sort=False)
    
    return userDF

In [ ]:
def extractDate(timestamp):
    return timestamp.date()

In [ ]:
def removePeriod(phrase):
    return phrase.strip('..')

In [ ]:
def causalImpactPreprocessingMoodle(df):
    df = pd.merge(df, getUserList(), left_on='User full name', right_on='id')
    df.drop(['Affected user', 'Event context', 'Component', 'Origin', 'IP address', 'Description', 'User full name', 'id'], inplace = True, axis = 1)
    df['Event name'] = df['Event name'].apply(removePeriod)
    df['Time'] = df['Time'].apply(dateutil.parser.parse, dayfirst=False)
    df['Date'] = df['Time'].apply(extractDate)
    dfAgg = df.groupby(['Date', 'User Type', 'Event name'])['Time'].count()
    return dfAgg.to_frame(name = 'Total').reset_index()

In [ ]:
def causalImpactPreprocessingMoodleAllLogs(df):
    df = pd.merge(df, getUserList(), left_on='User full name', right_on='id')
    df.drop(['Affected user', 'Event context', 'Component', 'Origin', 'IP address', 'Description', 'User full name', 'id', 'Event name'], inplace = True, axis = 1)
    df['Time'] = df['Time'].apply(dateutil.parser.parse, dayfirst=False)
    df['Date'] = df['Time'].apply(extractDate)
    dfAgg = df.groupby(['Date', 'User Type'])['Time'].count()
    return dfAgg.to_frame(name = 'Total').reset_index()

In [ ]:
def causalImpactPreprocessingMoodleUniqueLogIns(df):
    df = pd.merge(df, getUserList(), left_on='User full name', right_on='id')
    df.drop(['Affected user', 'Event context', 'Component', 'Origin', 'IP address', 'Description', 'User full name'], inplace = True, axis = 1)
    df['Event name'] = df['Event name'].apply(removePeriod)
    df = pd.DataFrame(df.loc[df['Event name'] == 'User has logged in']).reset_index()
    df.drop(['index', 'Event name'], inplace = True, axis = 1)
    df['Time'] = df['Time'].apply(dateutil.parser.parse, dayfirst=False)
    df['Date'] = df['Time'].apply(extractDate)
    dfAgg = df.groupby(['Date', 'User Type', 'id'])['Time'].count()
    df = dfAgg.to_frame(name = 'Total').reset_index()
    df.drop(['id'], inplace = True, axis = 1)
    dfAgg = df.groupby(['Date', 'User Type'])['Total'].count()
    return dfAgg.to_frame(name = 'Total').reset_index()

In [ ]:
def causalImpactPreprocessingMoodleComponent(df):
    df = pd.merge(df, getUserList(), left_on='User full name', right_on='id')
    df.drop(['Affected user', 'Event context', 'Origin', 'IP address', 'Description', 'User full name', 'id'], inplace = True, axis = 1)
    df['Event name'] = df['Event name'].apply(removePeriod)
    df['Time'] = df['Time'].apply(dateutil.parser.parse, dayfirst=False)
    df['Date'] = df['Time'].apply(extractDate)
    dfAgg = df.groupby(['Date', 'User Type', 'Component', 'Event name'])['Time'].count()
    return dfAgg.to_frame(name = 'Total').reset_index()

In [ ]:
def causalImpactPreprocessingMoodleComponentOnly(df):
    df = pd.merge(df, getUserList(), left_on='User full name', right_on='id')
    df.drop(['Affected user', 'Event context', 'Event name', 'Origin', 'IP address', 'Description', 'User full name', 'id'], inplace = True, axis = 1)
    df['Time'] = df['Time'].apply(dateutil.parser.parse, dayfirst=False)
    df['Date'] = df['Time'].apply(extractDate)
    dfAgg = df.groupby(['Date', 'User Type', 'Component'])['Time'].count()
    return dfAgg.to_frame(name = 'Total').reset_index()

In [ ]:
def mapEventNameComponent(df):
    df = pd.merge(df, getUserList(), left_on='User full name', right_on='id')
    df.drop(['Time', 'Affected user', 'Event context', 'Origin', 'IP address', 'Description', 'User full name', 'id'], inplace = True, axis = 1)
    dfAgg = df.groupby(['Component', 'Event name'])['User Type'].count()
    return dfAgg.to_frame(name = 'Total').reset_index()